# Bonus Challenge 2 - ANOVA

In statistics, **Analysis of Variance (ANOVA)** is also used to analyze the differences among group means. The difference between t-test and ANOVA is the former is ued to compare two groups whereas the latter is used to compare three or more groups. [Read more about the difference between t-test and ANOVA](http://b.link/anova24).

From the ANOVA test, you receive two numbers. The first number is called the **F-value** which indicates whether your null-hypothesis can be rejected. The critical F-value that rejects the null-hypothesis varies according to the number of total subjects and the number of subject groups in your experiment. In [this table](http://b.link/eda14) you can find the critical values of the F distribution. **If you are confused by the massive F-distribution table, don't worry. Skip F-value for now and study it at a later time. In this challenge you only need to look at the p-value.**

The p-value is another number yielded by ANOVA which already takes the number of total subjects and the number of experiment groups into consideration. **Typically if your p-value is less than 0.05, you can declare the null-hypothesis is rejected.**

In this challenge, we want to understand whether there are significant differences among various types of pokemons' `Total` value, i.e. Grass vs Poison vs Fire vs Dragon... There are many types of pokemons which makes it a perfect use case for ANOVA. Use Ironhack's database to load the pokemon data (db: pokemon, table: pokemon_stats). 

In [1]:
# Import libraries
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
# Load the data:
from google.colab import files
uploaded = files.upload()

Saving Pokemon.csv to Pokemon.csv


In [3]:
import io
pokemon = pd.read_csv(io.BytesIO(uploaded['Pokemon.csv']))
# Dataset is now stored in a Pandas Dataframe

pokemon

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


**To achieve our goal, we use three steps:**

1. **Extract the unique values of the pokemon types.**

1. **Select dataframes for each unique pokemon type.**

1. **Conduct ANOVA analysis across the pokemon types.**

#### First let's obtain the unique values of the pokemon types. These values should be extracted from Type 1 and Type 2 aggregated. Assign the unique values to a variable called `unique_types`.

*Hint: the correct number of unique types is 19 including `NaN`. You can disregard `NaN` in next step.*

In [4]:
# Your code here
unique_types = set(pokemon['Type 1']).union(set(pokemon['Type 2']))
print(unique_types)

len(unique_types) # you should see 19

{nan, 'Flying', 'Ghost', 'Poison', 'Psychic', 'Normal', 'Water', 'Steel', 'Fire', 'Grass', 'Rock', 'Fighting', 'Electric', 'Ground', 'Dark', 'Dragon', 'Ice', 'Bug', 'Fairy'}


19

#### Second we will create a list named `pokemon_totals` to contain the `Total` values of each unique type of pokemons.

Why we use a list instead of a dictionary to store the pokemon `Total`? It's because ANOVA only tells us whether there is a significant difference of the group means but does not tell which group(s) are significantly different. Therefore, we don't need know which `Total` belongs to which pokemon type.

*Hints:*

* Loop through `unique_types` and append the selected type's `Total` to `pokemon_groups`.
* Skip the `NaN` value in `unique_types`. `NaN` is a `float` variable which you can find out by using `type()`. The valid pokemon type values are all of the `str` type.
* At the end, the length of your `pokemon_totals` should be 18.

In [6]:
pokemon_totals = []

# Your code here

unique_types = [x for x in unique_types if type(x) == str]

pokemon_totals = []

for u in unique_types:
    filt = (pokemon['Type 1']==u) | (pokemon['Type 2']==u)
    all_total = pokemon.loc[filt, 'Total'].tolist()
    pokemon_totals.append(all_total)
print(pokemon_totals)

len(pokemon_totals) # you should see 18

[[534, 634, 395, 251, 349, 479, 579, 262, 442, 245, 455, 352, 310, 460, 500, 600, 540, 515, 615, 580, 580, 580, 600, 262, 442, 265, 390, 535, 405, 320, 470, 250, 340, 460, 390, 405, 430, 330, 465, 465, 680, 680, 395, 270, 430, 270, 430, 414, 456, 310, 490, 460, 600, 700, 680, 780, 245, 340, 485, 424, 244, 474, 348, 498, 505, 411, 345, 545, 515, 510, 520, 600, 264, 358, 488, 313, 425, 490, 401, 567, 305, 473, 428, 350, 510, 370, 510, 580, 580, 580, 580, 600, 600, 278, 382, 499, 411, 500, 245, 535, 680], [310, 405, 500, 600, 435, 236, 380, 480, 295, 455, 555, 295, 455, 348, 498, 495, 485, 525, 480, 440, 680, 680, 303, 483, 335, 480, 275, 370, 520, 303, 483, 325, 448, 520, 520, 309, 474, 335, 335, 335, 335, 494, 494, 494, 494, 600], [318, 405, 525, 625, 195, 205, 395, 495, 288, 438, 275, 365, 505, 273, 365, 505, 245, 455, 320, 395, 490, 305, 450, 300, 390, 490, 335, 515, 325, 500, 310, 405, 500, 600, 340, 490, 250, 390, 535, 430, 385, 400, 302, 467, 458, 280, 515, 329, 479, 330, 500, 300,

18

#### Now we run ANOVA test on `pokemon_totals`.

*Hints:*

* To conduct ANOVA, you can use `scipy.stats.f_oneway()`. Here's the [reference](http://b.link/scipy44).

* What if `f_oneway` throws an error because it does not accept `pokemon_totals` as a list? The trick is to add a `*` in front of `pokemon_totals`, e.g. `stats.f_oneway(*pokemon_groups)`. This trick breaks the list and supplies each list item as a parameter for `f_oneway`.

In [7]:
# Your code here
from scipy import stats
from scipy.stats import f_oneway
stats.f_oneway(*pokemon_totals)

F_onewayResult(statistic=6.6175382960055344, pvalue=2.6457458815984803e-15)

#### Interpret the ANOVA test result. Is the difference significant?

In [ ]:
## Your comment here

# null hypothesis of ANOVA: no significant difference in each groups
# p-value is less than 0.05, we can declare the null-hypothesis is rejected.
# so there is significant difference